In [33]:
# Import Libraries
import numpy as np
import cv2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import pandas as pd

In [34]:
df=pd.read_csv('movies_data.csv')
df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [35]:
df.shape

(4803, 24)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [15]:
df.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


# 1 Data preprocessing

In [36]:
df.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [37]:
# Fill any NaN values with an empty string
df['overview'] = df['overview'].fillna('')

In [38]:
# Replace missing values for 'genres', 'keywords', 'overview', 'cast', and 'director' with empty strings
df['genres'] = df['genres'].fillna('')
df['keywords'] = df['keywords'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('')

# Check if missing values have been handled correctly
print(df[['genres', 'keywords', 'overview', 'cast', 'director']].isnull().sum())


genres      0
keywords    0
overview    0
cast        0
director    0
dtype: int64


In [39]:
# Combine features to create a soup of words (genres, keywords, overview)
def create_soup(x):
    return ' '.join(x['genres']) + ' ' + str(x['keywords']) + ' ' + x['overview']

In [40]:
# Apply the function to create a soup
df['soup'] = df.apply(create_soup, axis=1)

In [41]:
# Initialize a TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [42]:
# Fit and transform the soup into a matrix
tfidf_matrix = tfidf.fit_transform(df['soup'])

In [43]:
# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [44]:
# Reset the index of the movies DataFrame and drop the old index to avoid conflicts
df = df.reset_index(drop=True)

# Construct a reverse mapping of indices and movie titles
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [47]:
# Function to get movie recommendations based on the title
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

In [48]:
# Example usage
recommended_movies = get_recommendations('Avatar')
print("Movies recommended for 'Avatar':")
print(recommended_movies)

Movies recommended for 'Avatar':
1914       Lifeforce
1352         Gattaca
1531       Moonraker
3604       Apollo 18
3730           Cargo
239          Gravity
1354    Space Chimps
4593        Trekkies
2198         Lockout
1951      Space Dogs
Name: title, dtype: object


In [51]:
# Create a weighted soup of features
def create_weighted_soup(x):
    genres = x['genres'] * 3  # More weight to genres
    keywords = x['keywords'] * 2
    overview = x['overview'] * 1
    cast = x['cast'] * 2
    director = x['director'] * 2
    return genres + ' ' + keywords + ' ' + overview + ' ' + cast + ' ' + director
df['soup'] = df.apply(create_weighted_soup, axis=1)

In [53]:
# Reset index and create reverse mapping
df = df.reset_index(drop=True)
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

# 2 contenet based filtering

In [54]:
# Vectorize the 'soup' column
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['soup'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get movie recommendations based on the content
def get_content_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10 recommendations
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]


In [58]:
movies_data=df

# 3 collabrative filtering , 4 Hybrid filtering

In [59]:
# Function to get genre-based recommendations
def get_genre_recommendations(title):
    movie_genres = movies_data.loc[indices[title], 'genres']
    genre_movies = movies_data[movies_data['genres'].str.contains(movie_genres)]
    
    # Sort genre-based movies by a dummy popularity metric (e.g., number of words in the overview)
    genre_movies = genre_movies.assign(popularity=genre_movies['overview'].apply(lambda x: len(str(x).split())))
    genre_movies = genre_movies.sort_values(by='popularity', ascending=False)
    return genre_movies['title'].head(10)

In [60]:
# Combining content-based and genre-based recommendations
def hybrid_recommendations(title, cosine_sim=cosine_sim):
    if title not in indices:
        return "Movie not found in the dataset."
    
    content_recs = get_content_recommendations(title, cosine_sim)
    genre_recs = get_genre_recommendations(title)
    
    # Combine recommendations and remove duplicates
    combined_recs = pd.concat([content_recs, genre_recs]).drop_duplicates().head(10)
    return combined_recs

In [62]:
# Example usage
movie_title = "Toy Story" # Define the movie title here
recommended_movies_hybrid = hybrid_recommendations(movie_title)
print(f"Hybrid recommended movies based on '{movie_title}':")
print(recommended_movies_hybrid)

Hybrid recommended movies based on 'Toy Story':
343                        Toy Story 2
42                         Toy Story 3
3754                             Scoop
1805    The Curse of the Jade Scorpion
837                         Free Birds
2217          Everyone Says I Love You
2065                 To Rome with Love
2479                        Radio Days
1191                    Small Soldiers
2314                               Big
Name: title, dtype: object


In [65]:
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']
display (combined_features)


0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800                                                  NaN
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802                                                  NaN
Length: 4803, dtype: object

In [70]:
# Handle missing values before vectorization
combined_features = combined_features.fillna('')  # Replace NaN with empty strings

# Converting the text data to feature vectors
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
display (feature_vectors.shape)
print (feature_vectors)


(4803, 15224)

  (0, 190)	0.08090095524077573
  (0, 254)	0.09142522652029957
  (0, 4663)	0.11171447631533424
  (0, 11934)	0.1046041583076602
  (0, 4807)	0.1046041583076602
  (0, 3242)	0.21789291434059044
  (0, 2683)	0.22025431464014397
  (0, 5163)	0.1631611136740382
  (0, 12634)	0.3389293937832792
  (0, 14578)	0.12856573602436738
  (0, 2827)	0.250701039154941
  (0, 12542)	0.21363802702712734
  (0, 4361)	0.2357246920948712
  (0, 13421)	0.06994634624184701
  (0, 14960)	0.12774865735426136
  (0, 9814)	0.09074452085495464
  (0, 10092)	0.26697815139974085
  (0, 11713)	0.1510689341105145
  (0, 14969)	0.23197161550137896
  (0, 15200)	0.2049730435099724
  (0, 11686)	0.21789291434059044
  (0, 12352)	0.20207428946148326
  (0, 14652)	0.19468808815325506
  (0, 12842)	0.15399431943691239
  (0, 7687)	0.2357246920948712
  :	:
  (4799, 9590)	0.22381163060110326
  (4799, 3350)	0.216126974928572
  (4799, 2084)	0.20529606107814724
  (4799, 4888)	0.216126974928572
  (4799, 14236)	0.23464254445152807
  (4799, 776)	0.2346

In [71]:
similarity = cosine_similarity(feature_vectors)
print  (similarity )


[[1.         0.07278008 0.038882   ... 0.         0.         0.        ]
 [0.07278008 1.         0.03343555 ... 0.         0.         0.        ]
 [0.038882   0.03343555 1.         ... 0.         0.05595075 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.05595075 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [73]:
# Print Shape Cosine Similarity
display(similarity.shape)


(4803, 4803)

In [74]:
# Enter the movie name to get Similarity 
movie_name = input(' Enter your favourite movie name : ')


 Enter your favourite movie name :  Fight Club


In [75]:
# Creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)


['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [76]:
# Length of the List 
len(list_of_all_titles)


4803

In [78]:
# Finding the close match for the movie name given by the user
import difflib # Import the difflib module

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Fight Club', 'Fight Valley', 'Flight']


In [80]:
# Display the close match 
close_match = find_close_match[0]
print(close_match)


Fight Club


In [81]:
# Finding the index of the movie with the title
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)


662


In [82]:
# Get the similarity row for the selected index 
# These are the similarity values for the movie entered by the user 
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)


[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.03210443815286494), (4, 0.0), (5, 0.07438942647430152), (6, 0.0), (7, 0.0), (8, 0.012452864221877986), (9, 0.0), (10, 0.0), (11, 0.038886734668916685), (12, 0.0), (13, 0.11134027488982967), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.018404323720144614), (22, 0.0), (23, 0.0), (24, 0.0033621671274995635), (25, 0.003626264018235969), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0721495373036444), (31, 0.0), (32, 0.1121656455893418), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.02397858018658337), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.054755698869854305), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.003019581221766841), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.003735274034921505), (61, 0.0), (62, 0.011954475855215568), (63, 0.0), (64, 0.0), (65, 0.029490793260072704), (66, 0.0), (67, 0.

In [83]:
# Length of Similarity Score 
len(similarity_score)


4803

In [84]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)


[(662, 1.0), (3250, 0.15473295072829835), (326, 0.12443611341937774), (105, 0.12246892341056419), (1771, 0.12140824969896663), (133, 0.11884547374736423), (583, 0.11840708199447829), (2517, 0.11743612241193124), (1587, 0.11734101709164108), (100, 0.11316667938447378), (32, 0.1121656455893418), (13, 0.11134027488982967), (1594, 0.11048000585700266), (4567, 0.10834039901217787), (1685, 0.10686833057699704), (1553, 0.10686164038109801), (278, 0.10641509869154314), (4320, 0.10579559377343643), (830, 0.10303395330404305), (4146, 0.10123249864776908), (248, 0.1011123659116343), (2461, 0.1004324212037457), (3279, 0.09998547828428414), (196, 0.0984406933212515), (2838, 0.09624231743783845), (72, 0.09554173232673771), (442, 0.09162261515311176), (1135, 0.0893200145189182), (2740, 0.08888998905574652), (112, 0.08877817063837116), (3057, 0.08811374449414996), (2381, 0.08686709655948097), (3571, 0.08666537483929765), (3283, 0.08384087915944727), (89, 0.08296580547090177), (1512, 0.0802975178588331

In [85]:
# Print the name of similar movies based on the index – Top 30 

print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1


Movies suggested for you : 

1 . Fight Club
2 . Much Ado About Nothing
3 . Cinderella
4 . Alice Through the Looking Glass
5 . The 51st State
6 . Dark Shadows
7 . Big Fish
8 . The King's Speech
9 . The Curse of the Were-Rabbit
10 . The Curious Case of Benjamin Button
11 . Alice in Wonderland
12 . The Lone Ranger
13 . Corpse Bride
14 . Conversations with Other Women
15 . Keeping the Faith
16 . Se7en
17 . Planet of the Apes
18 . Dead Man's Shoes
19 . Kill Bill: Vol. 2
20 . Highway
21 . Mr. & Mrs. Smith
22 . Moonrise Kingdom
23 . Prefontaine
24 . Megamind
25 . The Fault in Our Stars
26 . Suicide Squad
27 . Face/Off
28 . Lord of War
29 . The New Guy


In [86]:
# Consolidated code 
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1


 Enter your favourite movie name :  se7en


Movies suggested for you : 

1 . Se7en
2 . Seven Pounds
3 . Outbreak
4 . The Beyond
5 . Minority Report
6 . Y Tu Mamá También
7 . Capote
8 . The Good Night
9 . The Dead Girl
10 . The Curious Case of Benjamin Button
11 . Mortdecai
12 . Duets
13 . The Infiltrator
14 . Beyond the Sea
15 . Sea of Love
16 . Light Sleeper
17 . Iron Man
18 . Traffic
19 . L.A. Confidential
20 . Fight Club
21 . Zodiac
22 . The Pallbearer
23 . The Talented Mr. Ripley
24 . The League of Extraordinary Gentlemen
25 . Basic
26 . Mindhunters
27 . Sky Captain and the World of Tomorrow
28 . Zero Effect
29 . On the Outs


In [89]:
pip install flask


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/101.7 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/101.7 kB ? eta -:--:--
   --------------- ----------------------- 41.0/101.7 kB 279.3 kB/s eta 0:00:01
   -------------------------------------- 101.7/101.7 kB 486.7 kB/s eta 0:00:00


In [90]:
from flask import Flask, render_template, request

# Create a Flask app instance
app = Flask(__name__)

# Define the route for the home page
@app.route('/')
def home():
    return render_template('index.html')

# Define the route for the recommendations
@app.route('/recommend', methods=['POST'])
def recommend():
    movie_title = request.form['movie_title']
    recommendations = get_content_recommendations(movie_title)
    return render_template('index.html', movie_title=movie_title, recommendations=recommendations)

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\bossh\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
